In [5]:
import logging as lg
from keras.layers import Input, Dense
from keras.models import load_model,Model
from keras.callbacks import ModelCheckpoint,LambdaCallback
import json
from pathlib import Path
from keras.utils import to_categorical
from keras.datasets import mnist

/home/ogura/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [81]:
class KerasNetInterface:
    
    def __init__(self,trained_epochs,result_dir,batch_size,valid_rate):
        self.result_dir=result_dir
        self.batch_size=batch_size
        self.valid_rate=valid_rate
        model_path=result_dir/f"{self.create_flag()}/model_{trained_epochs:02d}.h5"
        model_path.parent.mkdir(exist_ok=True)
        if model_path.exists():
            self.trained_epochs=trained_epochs
            lg.info("Loading the trained model...")
            self.model=self.load_model(model_path)
            lg.info("Loaded.")
        else:
            lg.info("Not found such a trained model.")
            lg.info("Creating new model...")
            self.trained_epochs=0
            self.model=self.construct()
            lg.info("Finished.")
    
    
    def load_model(self,model_path):
        return load_model(str(model_path))
    
    def fit(self, X, y, epochs):
        if self.trained_epochs >= epochs:
            lg.info("This model has already been traiend up to {trained_epochs}")
        callbacks=self.create_callbacks()
        self.model.fit(X,y,initial_epoch=self.trained_epochs,epochs=epochs,
                       batch_size=self.batch_size,callbacks=callbacks,validation_split=self.valid_rate)
        return self.model.history
    
    def save_model(self,save_path):
        self.model.save(str(save_path))
        
    def create_callbacks(self):
        model_path=self.result_dir / (self.create_flag()+"/model_{epoch:02d}.h5")
        mcp=ModelCheckpoint(str(model_path))
        hcp=LambdaCallback(on_epoch_end=lambda epoch,logs: self.save_history(epoch,logs))
        return [mcp,hcp]
    
    def save_history(self,epoch,logs):
        epoch+=1
        history_path=self.result_dir/ f"{self.create_flag()}/history_{epoch:02d}.h5"
        history=self.model.history.history
        if len(history)==0:
            history={k:[v] for k,v in logs.items()}
        else:
            for k,v in logs.items(): 
                history[k].append(v)
        with history_path.open("w") as f:
            json.dump(history,f)
        self.trained_epochs+=epoch
            
    def create_flag(self):
        pass
        
    def construct(self):
        pass
    
    def predict(self):
        pass
    

In [82]:
class KerasClassifierInterface(KerasNetInterface):
    metrics=["accuracy"]
    def predict(self,X):
        return np.argmax(self.model.predict(X),axis=1)
    
    def predict_proba(self,X):
        return np.argmax(self.model.predict(X),axis=1)

class KerasRegressorInterface(KerasNetInterface):
    metrics=["mse"]
    def predict(self,X):
        return self.model.predict(X)

In [83]:
class MLPClassifier(KerasClassifierInterface):
    
    def __init__(self,trained_epochs=0, result_dir=Path("result"),valid_rate=0.3, batch_size=256, input_dim=784,
                             med1_dim=300,med2_dim=100,output_dim=10,activation="relu",
                             loss='categorical_crossentropy',optimizer='rmsprop'):
        self.loss=loss
        self.optimizer=optimizer
        self.input_dim=input_dim
        self.med1_dim=med1_dim
        self.med2_dim=med2_dim
        self.output_dim=output_dim
        self.activation=activation
        super().__init__(trained_epochs,result_dir,batch_size,valid_rate)
        
    def construct(self):
        inputs = Input(shape=(self.input_dim,))
        x = Dense(self.med1_dim, activation=self.activation)(inputs)
        x = Dense(self.med2_dim, activation=self.activation)(x)
        predictions = Dense(self.output_dim, activation='softmax')(x)
        model = Model(inputs=inputs, outputs=predictions)
        model.compile(optimizer=self.optimizer, loss=self.loss,metrics=self.metrics)
        return model
    
    def create_flag(self):
        return f"{self.input_dim}_{self.med1_dim}_{self.med2_dim}_{self.output_dim}_{self.activation}_{self.batch_size}"
    

In [84]:
def reshape_mnist(x,y):
    sample_num=x.shape[0]
    x=x.reshape(sample_num, -1)
    x= x.astype('float32')
    x /= 255
    y = to_categorical(y, 10)
    return x,y

In [85]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train,y_train=reshape_mnist(x_train,y_train)
x_test,y_test=reshape_mnist(x_test,y_test)

In [86]:
model=MLPClassifier()

In [87]:
hisotry=model.fit(x_train,y_train,epochs=10)

Train on 42000 samples, validate on 18000 samples
Epoch 1/10
42000/42000 [==============================] - 1s 29us/step - loss: 0.3821 - acc: 0.8882 - val_loss: 0.1972 - val_acc: 0.9411
Epoch 2/10
42000/42000 [==============================] - 1s 21us/step - loss: 0.1472 - acc: 0.9563 - val_loss: 0.1456 - val_acc: 0.9557
Epoch 3/10
42000/42000 [==============================] - 1s 21us/step - loss: 0.0989 - acc: 0.9707 - val_loss: 0.1702 - val_acc: 0.9456
Epoch 4/10
42000/42000 [==============================] - 1s 21us/step - loss: 0.0723 - acc: 0.9778 - val_loss: 0.1474 - val_acc: 0.9525
Epoch 5/10
42000/42000 [==============================] - 1s 21us/step - loss: 0.0536 - acc: 0.9831 - val_loss: 0.1064 - val_acc: 0.9696
Epoch 6/10
42000/42000 [==============================] - 1s 21us/step - loss: 0.0412 - acc: 0.9877 - val_loss: 0.1283 - val_acc: 0.9636
Epoch 7/10
42000/42000 [==============================] - 1s 22us/step - loss: 0.0317 - acc: 0.9905 - val_loss: 0.1147 - val_acc